In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd

from obspy import Trace, Stream, UTCDateTime
import obsplus
from obsplus.utils.time import to_utc

from mopy import StatsGroup, TraceGroup, SpectrumGroup

t, a, b, c, f = sym.symbols('t a b c f')

In [ ]:
# Get gausian and derivative/integral
gaus = a * sym.exp((-(t - b)**2) / (2 * c **2))
d_gaus = sym.diff(gaus, t)
int_gaus = sym.integrate(gaus, t)

In [ ]:
# get freq domain
F_gaus = sym.fourier_transform(gaus, t, f).rewrite(sym.Integral)
F_d_gaus = sym.fourier_transform(d_gaus, t, f).rewrite(sym.Integral)
F_int_gaus = sym.fourier_transform(int_gaus, t, f).rewrite(sym.Integral)

In [ ]:
get_gaus = sym.lambdify([t, a, b, c], gaus)
get_d_gaus = sym.lambdify([t, a, b, c], d_gaus)
get_int_gaus = sym.lambdify([t, a, b, c], int_gaus)

In [ ]:
t1, t2, dt = 0, 10.24, 0.01
a, b, c = 0.1, 5, np.sqrt(3)
x_orig = np.arange(t1, t2+1, dt)
x = x_orig
# get source in 1) displacement 2) velocity
# dt = 0.01
# a, b, c = 0.1, 5, np.sqrt(2)
# # x = np.arange(0, 10/2, dt)
# x = np.arange(0, 10, dt)
ns = {}

# Without zero-padding
# source_disp_st = 
source_disp = np.zeros(len(x))
source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
ns["source"] = len(x)
source_vel = np.zeros(len(x))
source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
# With zero padding and a serious discontinuity
# x = np.arange(t1, (t2+1)/2, dt)
# dis_source_disp = np.zeros(len(x)*2)
# dis_source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
# dis_source_vel = np.zeros(len(x)*2)
# dis_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
x = np.arange(t1, (t2+1)*0.75, dt)
ns["discosninuity"] = len(x)
dis_source_disp = np.zeros(len(x_orig))
dis_source_disp[0:len(x)] = get_gaus(x, a=a, b=b, c=c)  # displacement
dis_source_vel = np.zeros(len(x_orig))
dis_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b, c=c)  # velocity
# With zero padding, but relatively continuous
# x = np.arange(t1, (t2+1)/2, dt)
# zp_source_disp = np.zeros(len(x)*2)
# zp_source_disp[0:len(x)] = get_gaus(x, a=a, b=b/2, c=c/2)  # displacement
# zp_source_vel = np.zeros(len(x)*2)
# zp_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b/2, c=c/2)  # velocity
x = np.arange(t1, (t2+1)*0.75, dt)
ns["zero_padded"] = len(x)
zp_source_disp = np.zeros(len(x_orig))
zp_source_disp[0:len(x)] = get_gaus(x, a=a, b=b*0.75, c=c*0.75)  # displacement
zp_source_vel = np.zeros(len(x_orig))
zp_source_vel[0:len(x)] = get_d_gaus(x, a=a, b=b*0.75, c=c*0.75)  # velocity

x = x_orig

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_disp)
ax2.plot(x, dis_source_disp)
ax3.plot(x, zp_source_disp)
# plt.plot(x, source_disp)
ax1.set_ylabel('displacement amplitude (m)')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
ax1.plot(x, source_vel)
ax2.plot(x, dis_source_vel)
ax3.plot(x, zp_source_vel)
ax1.set_ylabel('velocity amplitude (m/s)')

In [ ]:
ds = obsplus.load_dataset("coal_node")
# Load up some events
cat = ds.event_client.get_events()
# Load up a station inventory
inv = ds.station_client.get_stations()
# Generate a StatsGroup that can be used to create another StatsGroup
sg = StatsGroup(catalog=cat, inventory=inv)

# Two streams, one has the full data, the other has only part of it
# (to see the effects of zero padding the data)
df_contents = {
    "event_id": ["event1", "event1", "event1"],
    "seed_id": ["UK.STA1..HHZ", "UK.STA2..HHZ", "UK.STA3..HHZ"],
    "seed_id_less": ["UK.STA1..HH", "UK.STA2..HH", "UK.STA3..HH"],
    "phase_hint": ["P", "P", "P"],
    "time": [np.datetime64("2020-01-01T00:00:01"), np.datetime64("2020-01-01T00:00:01"), np.datetime64("2020-01-01T00:00:01")],
    "starttime": [np.datetime64("2020-01-01T00:00:00"), np.datetime64("2020-01-01T00:00:00"), np.datetime64("2020-01-01T00:00:00"),],
    "endtime": [np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2*1000), 'ms') - np.timedelta64(int(dt*1000), 'ms'), 
                np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2//2*1000), 'ms') - np.timedelta64(int(dt*1000), 'ms'),
                np.datetime64("2020-01-01T00:00:00") + np.timedelta64(int(t2//2*1000), "ms") - np.timedelta64(int(dt*1000), "ms")
               ],
    "ray_path_length_m": [2000, 2000, 2000],
    "hyp_distance_m": [2000, 2000, 2000],
}
df = pd.DataFrame(df_contents, columns=df_contents.keys())
df.set_index(['phase_hint', 'event_id', 'seed_id_less', 'seed_id'], inplace=True)
df["sampling_rate"] = 1/dt
sg = sg.new_from_dict(data=df)  # This is a weird way to do this

st = Stream()
tr = Trace(data=source_disp, header={
    "starttime": to_utc(sg.data.iloc[0].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA1",
    "channel": "HHZ",
})
st.append(tr)
tr = Trace(data=dis_source_disp, header={
    "starttime": to_utc(sg.data.iloc[1].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA2",
    "channel": "HHZ",
})
st.append(tr)
tr = Trace(data=zp_source_disp, header={
    "starttime": to_utc(sg.data.iloc[2].starttime),
    "delta": dt,
    "network": "UK",
    "station": "STA3",
    "channel": "HHZ",
})
st.append(tr)
tg = TraceGroup(sg, st.copy(), "displacement").fillna()  # START HERE... MAKE SURE TO CREATE THE STREAM BEFORE ADDING THE ZERO PADDING TO THE END TO ENSURE WE'RE GETTING THE RIGHT SAMPLE LENGTHS? BUT IT STILL SEEMS LIKE THERE WOULD BE A SAMPLE LENGTH DEPENDENCE THAT IS NOT ACCOUNTED FOR...

spec = tg.dft()
spec_mtspec = tg.mtspec()

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(15, 10))
ax1.loglog(spec.data.columns, spec.abs().data.iloc[0])
ax1.loglog(spec_mtspec.data.columns, spec_mtspec.to_spectra_type("dft").data.iloc[0])
ax4.loglog(spec.data.columns, spec.to_spectra_type("psd").abs().data.iloc[0])
ax4.loglog(spec_mtspec.data.columns, spec_mtspec.data.iloc[0])

ax2.loglog(spec.data.columns, spec.abs().data.iloc[1])
ax2.loglog(spec_mtspec.data.columns, spec_mtspec.to_spectra_type("dft").data.iloc[1])
ax5.loglog(spec.data.columns, spec.to_spectra_type("psd").abs().data.iloc[1])
ax5.loglog(spec_mtspec.data.columns, spec_mtspec.data.iloc[1])

ax3.loglog(spec.data.columns, spec.abs().data.iloc[2])
ax3.loglog(spec_mtspec.data.columns, spec_mtspec.to_spectra_type("dft").data.iloc[2])
ax6.loglog(spec.data.columns, spec.to_spectra_type("psd").abs().data.iloc[2])
ax6.loglog(spec_mtspec.data.columns, spec_mtspec.data.iloc[2])